#Урок 4. Парсинг HTML. XPath
Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath.
Структура данных должна содержать:
 - название источника;
 - наименование новости;
 - ссылку на новость;
 - дата публикации.
Сложить собранные данные в БД
Минимум один сайт, максимум - все три

In [35]:
from lxml import html
import pandas as pd
import requests
from pprint import pprint
from pymongo import MongoClient
from datetime import date, datetime
curr_date= datetime.today().strftime("%Y-%m-%d")
print('Сегодня: ', curr_date)

Сегодня:  2021-08-30


In [40]:
header = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.18 Safari/537.36 OPR/79.0.4143.3 (Edition beta)"
}

response = requests.get('https://yandex.ru/news', headers=header)

dom = html.fromstring(response.text)

items = dom.xpath('//*/article') #('//*[@class="mg-card__text"] | //*[@class="mg-card__inner"]')
print(len(items))
items_list = []

for item in items:
    items_data = {}
    name = item.xpath('.//h2[@class="mg-card__title"]//text()')[0].replace('\xa0— 2','').replace('\xa0',' ')
    source = item.xpath('.//a[contains(@class,"mg-card__source-link")]/text()')
    link = item.xpath('.//a[@class="mg-card__link"]/@href')
    time = item.xpath(".//span[@class='mg-card-source__time']//text()")

    items_data['name'] = name
    items_data['source'] = source[0]
    items_data['link'] = link[0]

    if time[0].split()[0] == 'Вчера':
        time = datetime.strptime((curr_date+' '+time[0].split()[-1]), "%Y-%m-%d %H:%M")-timedelta(days=1)
    else:
	    time = datetime.strptime((curr_date+' '+time[0].split()[-1]), "%Y-%m-%d %H:%M")

    # time = datetime.strptime(time[0].split()[-1], '%H:%M')
    items_data['date'] = time

    items_list.append(items_data)


65


In [41]:
df = pd.DataFrame(items_list)
df.head(10)

,name,source,link,date
0,Единовременную выплату в 10 тыс. рублей перечи...,ТАСС,https://yandex.ru/news/story/Edinovremennuyu_v...,2021-08-30 21:45:00
1,Цена газа в Европе достигла рекордных $600 за ...,Коммерсантъ,https://yandex.ru/news/story/Cena_gaza_vEvrope...,2021-08-30 21:50:00
2,МВД признало пребывание в России комика Идрака...,РИА Новости,https://yandex.ru/news/story/MVD_priznalo_preb...,2021-08-30 21:27:00
3,ТАСС: пресс-секретарь Алексея Навального Кира ...,Lenta.ru,https://yandex.ru/news/story/TASS_press-sekret...,2021-08-30 21:43:00
4,Глава МИД РФ Сергей Лавров: нападки на Сталина...,Lenta.ru,https://yandex.ru/news/story/Glava_MID_RF_Serg...,2021-08-30 21:01:00
5,Герой России Роман Шадрин умер в возрасте 54 лет,РИА Новости,https://yandex.ru/news/story/Geroj_Rossii_Roma...,2021-08-30 19:30:00
6,E1.RU опубликовало репортаж из реанимации глав...,Solenka.info,https://yandex.ru/news/story/E1.RU_opublikoval...,2021-08-30 17:30:00
7,Покрас Лампас создал арт-объект в Екатеринбург...,ТАСС,https://yandex.ru/news/story/Pokras_Lampas_soz...,2021-08-30 21:25:00
8,РМК построит в Екатеринбурге музей оружия по п...,e1.ru,https://yandex.ru/news/story/RMK_postroit_vEka...,2021-08-30 20:38:00
9,СК: змея укусила ребенка в контактном зоопарке...,РИА Новости,https://yandex.ru/news/story/SK_zmeya_ukusila_...,2021-08-30 21:38:00


In [42]:
client = MongoClient('mongodb://localhost:27017/')
news = client.news
db = news.news

for i in range(0, len(df)):
	# db.insert_one(data.loc[i].to_dict())
	db.update_one({'link': df.loc[i].to_dict()['link']}, {
		'$set': df.loc[i].to_dict()}, upsert=True)